In [1]:
# Validate the best model by Hyperparameter tune 

import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, SimpleRNN, LSTM, Conv1D, MaxPooling1D, Flatten, Input
from sklearn.model_selection import ParameterGrid
import tensorflow as tf
import os
import random

# Function to reset random seeds
def reset_random_seeds():
    os.environ['PYTHONHASHSEED'] = str(2)
    tf.random.set_seed(2)
    np.random.seed(2)
    random.seed(2)

# Reset seeds for reproducibility
reset_random_seeds()

# Load data
TOTALGP = pd.read_csv('C:/Allan Folder/ALLAN - AMBY/Emily/INSURANCE FORECAST/FINAL DATASET/OLR asof March2020/GPDATASET.csv')

# Trim spaces from column names
TOTALGP.columns = TOTALGP.columns.str.strip()

# Convert 'MONTHSALES' column to datetime and set as index
TOTALGP['MONTHSALES'] = pd.to_datetime(TOTALGP['MONTHSALES'], format='%Y%m')
TOTALGP.set_index('MONTHSALES', inplace=True)

TOTALGP_column_name = 'TOTALRL'

# Handle missing values
TOTALGP = TOTALGP.ffill().bfill()

# Normalize the data
scaler = MinMaxScaler()
TOTALGP[TOTALGP_column_name] = scaler.fit_transform(TOTALGP[[TOTALGP_column_name]])

# Prepare the data
train_size = int(0.8 * len(TOTALGP))
train_data, test_data = TOTALGP[TOTALGP_column_name][:train_size], TOTALGP[TOTALGP_column_name][train_size:]

def create_sequences(data, seq_length):
    sequences = []
    target = []
    for i in range(len(data) - seq_length):
        sequences.append(data[i:i+seq_length])
        target.append(data[i+seq_length])
    return np.array(sequences), np.array(target)

seq_length = 12
X_train, y_train = create_sequences(train_data.values, seq_length)
X_test, y_test = create_sequences(test_data.values, seq_length)

# Reshape for RNN, LSTM, CNN (not needed for DNN)
X_train_rnn = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test_rnn = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

from tensorflow.keras.layers import Input

# Define model building functions
def build_dnn(units1, units2):
    model = Sequential()
    model.add(Input(shape=(seq_length,)))
    model.add(Dense(units1, activation='relu'))
    model.add(Dense(units2, activation='relu'))
    model.add(Dense(1))  # Output layer for regression
    model.compile(optimizer='adam', loss='mse')
    return model

def build_rnn(units):
    model = Sequential()
    model.add(Input(shape=(seq_length, 1)))
    model.add(SimpleRNN(units, activation='relu'))
    model.add(Dense(1))  # Output layer for regression
    model.compile(optimizer='adam', loss='mse')
    return model

def build_lstm(units):
    model = Sequential()
    model.add(Input(shape=(seq_length, 1)))
    model.add(LSTM(units, activation='relu'))
    model.add(Dense(1))  # Output layer for regression
    model.compile(optimizer='adam', loss='mse')
    return model

def build_cnn(filters, kernel_size):
    model = Sequential()
    model.add(Input(shape=(seq_length, 1)))
    model.add(Conv1D(filters=filters, kernel_size=kernel_size, activation='relu'))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Flatten())
    model.add(Dense(50, activation='relu'))
    model.add(Dense(1))  # Output layer for regression
    model.compile(optimizer='adam', loss='mse')
    return model

# Define hyperparameter grid
param_grid = {
    'DNN': {
        'units1': [64, 128],
        'units2': [32, 64],
        'epochs': [100, 300],
        'batch_size': [16, 32]
    },
    'RNN': {
        'units': [30, 50],
        'epochs': [100, 300],
        'batch_size': [16, 32]
    },
    'LSTM': {
        'units': [30, 50],
        'epochs': [100, 300],
        'batch_size': [16, 32]
    },
    'CNN': {
        'filters': [32, 64],
        'kernel_size': [3, 5],
        'epochs': [100, 300],
        'batch_size': [16, 32]
    }
}

# Store RMSE scores
rmse_scores = {}

# Iterate through hyperparameter combinations
for name, params in param_grid.items():
    grid = ParameterGrid(params)
    for param_comb in grid:
        print(f"Testing {name} with params: {param_comb}")
        if name == 'DNN':
            model = build_dnn(param_comb['units1'], param_comb['units2'])
        elif name == 'RNN':
            model = build_rnn(param_comb['units'])
        elif name == 'LSTM':
            model = build_lstm(param_comb['units'])
        elif name == 'CNN':
            model = build_cnn(param_comb['filters'], param_comb['kernel_size'])
        
        model.fit(X_train if name == 'DNN' else X_train_rnn, y_train,
                  epochs=param_comb['epochs'], batch_size=param_comb['batch_size'],
                  validation_data=(X_test if name == 'DNN' else X_test_rnn, y_test), verbose=2)
        y_pred = model.predict(X_test if name == 'DNN' else X_test_rnn)
        
        rmse_test = np.sqrt(mean_squared_error(y_test, y_pred.flatten()))
        rmse_scores[f"{name}_params_{param_comb}"] = rmse_test
        
        print(f"{name} with params {param_comb} - RMSE: {rmse_test}")

# Print all RMSE scores
print("All RMSE Scores:")
for model_name, rmse_score in rmse_scores.items():
    print(f"{model_name}: {rmse_score}")


Testing DNN with params: {'batch_size': 16, 'epochs': 100, 'units1': 64, 'units2': 32}
Epoch 1/100
8/8 - 1s - 117ms/step - loss: 0.0384 - val_loss: 0.0666
Epoch 2/100
8/8 - 0s - 7ms/step - loss: 0.0222 - val_loss: 0.0572
Epoch 3/100
8/8 - 0s - 6ms/step - loss: 0.0171 - val_loss: 0.0738
Epoch 4/100
8/8 - 0s - 6ms/step - loss: 0.0159 - val_loss: 0.0798
Epoch 5/100
8/8 - 0s - 6ms/step - loss: 0.0153 - val_loss: 0.0665
Epoch 6/100
8/8 - 0s - 8ms/step - loss: 0.0145 - val_loss: 0.0635
Epoch 7/100
8/8 - 0s - 8ms/step - loss: 0.0138 - val_loss: 0.0687
Epoch 8/100
8/8 - 0s - 7ms/step - loss: 0.0134 - val_loss: 0.0693
Epoch 9/100
8/8 - 0s - 7ms/step - loss: 0.0132 - val_loss: 0.0666
Epoch 10/100
8/8 - 0s - 8ms/step - loss: 0.0128 - val_loss: 0.0668
Epoch 11/100
8/8 - 0s - 7ms/step - loss: 0.0124 - val_loss: 0.0678
Epoch 12/100
8/8 - 0s - 7ms/step - loss: 0.0122 - val_loss: 0.0673
Epoch 13/100
8/8 - 0s - 6ms/step - loss: 0.0119 - val_loss: 0.0668
Epoch 14/100
8/8 - 0s - 6ms/step - loss: 0.0117 -

In [6]:
# Initialize dictionaries to store the best RMSE for each model type
best_rmse = {'DNN': float('inf'), 'RNN': float('inf'), 'LSTM': float('inf'), 'CNN': float('inf')}
best_params = {'DNN': None, 'RNN': None, 'LSTM': None, 'CNN': None}

# Extract the best RMSE and corresponding parameters for each model type
for model_name, rmse_score in rmse_scores.items():
    # Determine model type from the model name
    if model_name.startswith('DNN'):
        model_type = 'DNN'
    elif model_name.startswith('RNN'):
        model_type = 'RNN'
    elif model_name.startswith('LSTM'):
        model_type = 'LSTM'
    elif model_name.startswith('CNN'):
        model_type = 'CNN'
    else:
        continue  # Skip if model type is unknown
    
    # Update best RMSE and parameters if the current score is better
    if rmse_score < best_rmse[model_type]:
        best_rmse[model_type] = rmse_score
        best_params[model_type] = model_name

# Print the best model for each type
print("Best Model for Each Architecture:")
for model_type, params_name in best_params.items():
    print(f"{model_type}: {params_name} - RMSE: {best_rmse[model_type]}")

# Identify the overall best model
overall_best_model = min(best_rmse, key=best_rmse.get)
print(f"\nOverall Best Model: {best_params[overall_best_model]} - RMSE: {best_rmse[overall_best_model]}")


Best Model for Each Architecture:
DNN: DNN_params_{'batch_size': 16, 'epochs': 300, 'units1': 128, 'units2': 64} - RMSE: 0.2227522141760652
RNN: RNN_params_{'batch_size': 16, 'epochs': 300, 'units': 50} - RMSE: 0.22930139554290346
LSTM: LSTM_params_{'batch_size': 16, 'epochs': 100, 'units': 30} - RMSE: 0.24622424068080687
CNN: CNN_params_{'batch_size': 16, 'epochs': 300, 'filters': 64, 'kernel_size': 3} - RMSE: 0.22463940635538335

Overall Best Model: DNN_params_{'batch_size': 16, 'epochs': 300, 'units1': 128, 'units2': 64} - RMSE: 0.2227522141760652


In [7]:
# Print all RMSE scores
print("All RMSE Scores:")
for model_name, rmse_score in rmse_scores.items():
    print(f"{model_name}: {rmse_score}")

All RMSE Scores:
DNN_params_{'batch_size': 16, 'epochs': 100, 'units1': 64, 'units2': 32}: 0.23803716199416994
DNN_params_{'batch_size': 16, 'epochs': 100, 'units1': 64, 'units2': 64}: 0.2596783525452657
DNN_params_{'batch_size': 16, 'epochs': 100, 'units1': 128, 'units2': 32}: 0.22990127722531994
DNN_params_{'batch_size': 16, 'epochs': 100, 'units1': 128, 'units2': 64}: 0.2765078181039247
DNN_params_{'batch_size': 16, 'epochs': 300, 'units1': 64, 'units2': 32}: 0.3019283734671018
DNN_params_{'batch_size': 16, 'epochs': 300, 'units1': 64, 'units2': 64}: 0.26319733902316006
DNN_params_{'batch_size': 16, 'epochs': 300, 'units1': 128, 'units2': 32}: 0.266847941577415
DNN_params_{'batch_size': 16, 'epochs': 300, 'units1': 128, 'units2': 64}: 0.2227522141760652
DNN_params_{'batch_size': 32, 'epochs': 100, 'units1': 64, 'units2': 32}: 0.23827585231368928
DNN_params_{'batch_size': 32, 'epochs': 100, 'units1': 64, 'units2': 64}: 0.25667931691566204
DNN_params_{'batch_size': 32, 'epochs': 100, 